<a href="https://colab.research.google.com/github/UnknownNobody10/Fall2024/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step 1: Install Dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

#Step 2: Add environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

#these are new enviromental variables
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"
#Step 3: Initialize Pyspark
import findspark
findspark.init()

In [ ]:
#creating spark context
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

In [ ]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.GraphLoader
import org.apache.spark.graphx.Graph

val vertices: RDD[(VertexId, (String, Int))] = sc.parallelize(Seq(
    (1L, ("Aron", 350)),
    (2L, ("Bill", 360)),
    (3L, ("Claire", 195)),
    (4L, ("Daniel", 90)),
    (5L, ("Eric", 90)),
    (6L, ("Frank", 215)),
    (7L, ("Graham", 30)),
    (8L, ("Henry", 25)),
    (9L, ("Inna", 25)),
    (10L, ("Jen", 20))
))

val edges: RDD[Edge[Int]] = sc.parallelize(Seq(
    Edge(1L, 2L, 60),
    Edge(2L, 1L, 50),
    Edge(1L, 3L, 50),
    Edge(3L, 1L, 100),
    Edge(1L, 4L, 90),
    Edge(3L, 9L, 25),
    Edge(3L, 10L, 20),
    Edge(2L, 6L, 50),
    Edge(6L, 2L, 110),
    Edge(6L, 7L, 30),
    Edge(6L, 8L, 25),
    Edge(2L, 5L, 90)
))

val graph: Graph(vertices, edges)


# Print the vertices and edges
println("Vertices: ")
graph.vertices.collect.foreach {case (id, (name, total_seconds)) =>
  println(s"Vertex ID: $id, Name: $name, Total Seconds: $total_seconds")
}

println("\nEdges: ")
graph.edges.collect.foreach { edge =>
  println(s"Source: $(edge.srcID}, Destination: ${edge.dstID}, Seconds: ${edge.attr}")
}

#degrees, in-degrees, out-degrees
val degrees = graph.degrees
val inDegrees = graph.inDegrees
val outDegrees = graph.outDegrees

#BFS
val frankId = vertices.filter { case (_, (name, _)) => name == "Frank" }.first()._1
val targetVertices = vertices.filter { case (_, (_, seconds)) => seconds > 215}.map(_._1).collect.toSet
val bfs = graph.bfs(frankId, targetVertices)
